In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
pd.set_option('display.max_columns', None)

main_path = '/Users/PC/Desktop/Insight/MTASafety'

In [4]:
#Loading crime data
df = pd.read_csv(main_path + '/CrimeData/CrimeData10.csv', low_memory=False)

In [5]:
# For crime data concatenating date and time and convert to datetime object
df['datetime'] = df['CMPLNT_FR_DT'] + ' ' + df['CMPLNT_FR_TM']
df['datetime_clean'] = [datetime.strptime(x, '%m/%d/%Y %H:%M:%S') for x in df['datetime']]

In [6]:
# For crime data adding additional datetime columns
df['year'] = [x.year for x in df['datetime_clean']]
df['weekday'] = df[['datetime_clean']].apply(lambda x: datetime.strftime(x['datetime_clean'], '%A'), axis=1)
df['week'] = [x.isocalendar()[1] for x in df['datetime_clean']]
df['hour'] = [x.hour for x in df['datetime_clean']]

In [8]:
#Creatinging dictionaries for additional variables
def timebin(hour):
    if hour ==0:
        return 6
    if hour <= 4:
        return 1
    if hour <=8:
        return 2
    if hour <=12:
        return 3
    if hour <= 16:
        return 4
    if hour <= 20:
        return 5
    if hour <= 24:
        return 6
    
hourgroups = {6:'8pm - 12am', 
              1: '12am - 4am', 
              2:'4am - 8am', 
              3:'8am - 12pm', 
              4:'12pm - 4pm', 
              5:'4pm - 8pm'}

wkdaynbr = {'Friday': 5,
 'Monday': 1,
 'Saturday': 6,
 'Sunday': 0,
 'Thursday': 4,
 'Tuesday': 2,
 'Wednesday': 3}

In [9]:
#Creating additional variables
df['timegroup'] = df['hour'].apply(timebin)
df['timegroupstr'] = df['timegroup'].map(hourgroups)
df['wkdaynbr'] = df['weekday'].map(wkdaynbr)

In [11]:
#Creating an identity variable to group on
df['Ident']= df['Station'].astype(str) + df['year'].astype(str) + df['week'].astype(str) + df['wkdaynbr'].astype(str) + df['timegroup'].astype(str)

In [13]:
#Creating grouped variables to merge
df['Complaint']=1
dfsums=df.groupby('Ident')['Complaint'].sum().reset_index()
dfcrimes = df.groupby(['Ident'])['OFNS_DESC'].apply(', '.join).reset_index()
dfcat = df.groupby(['Ident'])['LAW_CAT_CD'].apply(', '.join).reset_index()
dfeverything_else = df.groupby('Ident').first()

In [43]:
#Merging the grouped variables
dfcomb=pd.merge(dfsums, dfcrimes, on=['Ident'], how='left')
dfcomb2=pd.merge(dfcomb, dfcat, on=['Ident'], how='left')
dfcomb3=pd.merge(dfcomb2, dfeverything_else, on=['Ident'], how='left')

In [59]:
dfcomb3.head()

,Ident,Complaint_x,OFNS_DESC_x,LAW_CAT_CD_x,Unnamed: 0,Unnamed: 0.1,CMPLNT_NUM,CMPLNT_FR_DT,DayofWeek,Month,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC_y,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD_y,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,ORIGINALROW,OBSOLETE_ID,LONG/LAT,Station,Distance,STATION_NAME.1,datetime,datetime_clean,year,weekday,week,hour,timegroup,timegroupstr,wkdaynbr,Complaint_y
0,1 AV_L20171432,1,ASSAULT 3 & RELATED OFFENSES,MISDEMEANOR,12,1853,855410583,4/5/2017,Wed,April,8:04:00,4/5/2017,8:17:00,13,4/5/2017,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,COMPLETED,MISDEMEANOR,MANHATTAN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"989,083","205,732",NaN,NaN,NaN,4.0,40.731370,-73.982563,"(40.73137039, -73.982563257)",PATROL BORO MAN SOUTH,1 AVENUE,25-44,BLACK,M,1.328472222,140285.0,"(-73.98256326, 40.73137039)",1 AV_L,0.054,1 AVENUE,4/5/2017 8:04:00,2017-04-05 08:04:00,2017,Wednesday,14,8,2,4am - 8am,3,1
1,1 AV_L20171753,1,SEX CRIMES,MISDEMEANOR,0,2258,727529072,4/28/2017,Fri,April,10:30:00,4/28/2017,12:00:00,13,4/28/2017,233,SEX CRIMES,175,"SEXUAL ABUSE 3,2",COMPLETED,MISDEMEANOR,MANHATTAN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"988,874","207,673",45-64,WHITE HISPANIC,M,4.0,40.736698,-73.983316,"(40.73669807, -73.983316026)",PATROL BORO MAN SOUTH,1 AVENUE,25-44,WHITE,F,1.615972222,168841.0,"(-73.98331603, 40.73669807)",1 AV_L,0.278,1 AVENUE,4/28/2017 10:30:00,2017-04-28 10:30:00,2017,Friday,17,10,3,8am - 12pm,5,1
2,1 AV_L20171765,1,SEX CRIMES,MISDEMEANOR,1,2283,172497442,4/29/2017,Sat,April,19:20:00,4/29/2017,19:25:00,13,5/15/2017,233,SEX CRIMES,175,"SEXUAL ABUSE 3,2",COMPLETED,MISDEMEANOR,MANHATTAN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"988,874","207,673",UNKNOWN,BLACK,M,4.0,40.736698,-73.983316,"(40.73669807, -73.983316026)",PATROL BORO MAN SOUTH,1 AVENUE,18-24,WHITE,F,1.848611111,170184.0,"(-73.98331603, 40.73669807)",1 AV_L,0.278,1 AVENUE,4/29/2017 19:20:00,2017-04-29 19:20:00,2017,Saturday,17,19,5,4pm - 8pm,6,1
3,1 AV_L20171822,1,SEX CRIMES,MISDEMEANOR,2,2351,983609181,5/2/2017,Tue,May,8:15:00,5/2/2017,8:25:00,13,5/3/2017,233,SEX CRIMES,175,"SEXUAL ABUSE 3,2",COMPLETED,MISDEMEANOR,MANHATTAN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"988,874","207,673",25-44,BLACK,M,4.0,40.736698,-73.983316,"(40.73669807, -73.983316026)",PATROL BORO MAN SOUTH,1 AVENUE,25-44,WHITE,F,1.678472222,174404.0,"(-73.98331603, 40.73669807)",1 AV_L,0.278,1 AVENUE,5/2/2017 8:15:00,2017-05-02 08:15:00,2017,Tuesday,18,8,2,4am - 8am,2,1
4,1 AV_L2017234,1,HARRASSMENT 2,VIOLATION,7,159,178632670,1/11/2017,Wed,January,15:00:00,1/11/2017,15:05:00,13,1/11/2017,578,HARRASSMENT 2,637,"HARASSMENT,SUBD 1,CIVILIAN",COMPLETED,VIOLATION,MANHATTAN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"989,083","205,732",18-24,BLACK,M,4.0,40.731370,-73.982563,"(40.73137039, -73.982563257)",PATROL BORO MAN SOUTH,1 AVENUE,45-64,WHITE,M,0.159027778,39323.0,"(-73.98256326, 40.73137039)",1 AV_L,0.054,1 AVENUE,1/11/2017 15:00:00,2017-01-11 15:00:00,2017,Wednesday,2,15,4,12pm - 4pm,3,1


In [60]:
dfcomb3.sort_values(by=['Complaint_x'])

,Ident,Complaint_x,OFNS_DESC_x,LAW_CAT_CD_x,Unnamed: 0,Unnamed: 0.1,CMPLNT_NUM,CMPLNT_FR_DT,DayofWeek,Month,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC_y,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD_y,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,ORIGINALROW,OBSOLETE_ID,LONG/LAT,Station,Distance,STATION_NAME.1,datetime,datetime_clean,year,weekday,week,hour,timegroup,timegroupstr,wkdaynbr,Complaint_y
0,1 AV_L20171432,1,ASSAULT 3 & RELATED OFFENSES,MISDEMEANOR,12,1853,855410583,4/5/2017,Wed,April,8:04:00,4/5/2017,8:17:00,13,4/5/2017,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,COMPLETED,MISDEMEANOR,MANHATTAN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"989,083","205,732",NaN,NaN,NaN,4.0,40.731370,-73.982563,"(40.73137039, -73.982563257)",PATROL BORO MAN SOUTH,1 AVENUE,25-44,BLACK,M,1.328472222,140285.0,"(-73.98256326, 40.73137039)",1 AV_L,0.054,1 AVENUE,4/5/2017 8:04:00,2017-04-05 08:04:00,2017,Wednesday,14,8,2,4am - 8am,3,1
10288,FORDHAM RD_BD20184534,1,GRAND LARCENY,FELONY,11010,13225,318508910,11/7/2018,Wed,November,16:35:00,11/7/2018,16:45:00,46,11/7/2018,109,GRAND LARCENY,439,"LARCENY,GRAND FROM OPEN AREAS, UNATTENDED",COMPLETED,FELONY,BRONX,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"1,012,539","253,107",UNKNOWN,UNKNOWN,U,11.0,40.861358,-73.897731,"(40.86135789900004, -73.89773096099998)",PATROL BORO BRONX,FORDHAM ROAD,45-64,BLACK,F,1:13241,886236.0,"(-73.89773096, 40.861357899999994)",FORDHAM RD_BD,0.004,FORDHAM ROAD,11/7/2018 16:35:00,2018-11-07 16:35:00,2018,Wednesday,45,16,4,12pm - 4pm,3,1
10289,FORDHAM RD_BD20184645,1,PETIT LARCENY,MISDEMEANOR,11011,13422,481367763,11/15/2018,Thu,November,17:00:00,NaN,NaN,46,12/18/2018,341,PETIT LARCENY,301,"LARCENY,PETIT BY ACQUIRING LOS",COMPLETED,MISDEMEANOR,BRONX,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"1,012,539","253,107",NaN,NaN,NaN,11.0,40.861358,-73.897731,"(40.86135789900004, -73.89773096099998)",PATROL BORO BRONX,FORDHAM ROAD,25-44,WHITE HISPANIC,M,1:14178,895404.0,"(-73.89773096, 40.861357899999994)",FORDHAM RD_BD,0.004,FORDHAM ROAD,11/15/2018 17:00:00,2018-11-15 17:00:00,2018,Thursday,46,17,5,4pm - 8pm,4,1
10290,FORDHAM RD_BD20184924,1,GRAND LARCENY,FELONY,11012,13837,230587946,12/4/2018,Tue,December,16:45:00,12/4/2018,16:47:00,46,12/4/2018,109,GRAND LARCENY,404,"LARCENY,GRAND FROM PERSON,PERSONAL ELECTRONIC ...",COMPLETED,FELONY,BRONX,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"1,012,539","253,107",UNKNOWN,UNKNOWN,M,11.0,40.861358,-73.897731,"(40.86135789900004, -73.89773096099998)",PATROL BORO BRONX,FORDHAM ROAD,18-24,WHITE HISPANIC,F,1:13845,917159.0,"(-73.89773096, 40.861357899999994)",FORDHAM RD_BD,0.004,FORDHAM ROAD,12/4/2018 16:45:00,2018-12-04 16:45:00,2018,Tuesday,49,16,4,12pm - 4pm,2,1
10291,FORDHAM RD_BD20184934,1,HARRASSMENT 2,VIOLATION,11013,13858,222634646,12/5/2018,Wed,December,14:00:00,12/5/2018,14:10:00,46,12/5/2018,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,BRONX,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"1,012,539","253,107",UNKNOWN,BLACK,U,11.0,40.861358,-73.897731,"(40.86135789900004, -73.89773096099998)",PATROL BORO BRONX,FORDHAM ROAD,18-24,WHITE HISPANIC,M,1:13866,918290.0,"(-73.89773096, 40.861357899999994)",FORDHAM RD_BD,0.004,FORDHAM ROAD,12/5/2018 14:00:00,2018-12-05 14:00:00,2018,Wednesday,49,14,4,12pm - 4pm,3,1
10292,FORDHAM RD_BD2018506,1,PETIT LARCENY,MISDEMEANOR,10989,7680,980828909,2/4/2018,Sun,February,21:30:00,2/4/2018,22:05:00,46,2/5/2018,341,PETIT LARCENY,339,"LARCENY,PETIT FROM OPEN AREAS,",COMPLETED,MISDEMEANOR,BRONX,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1,NaN,NaN,NaN,"1,012,539","253,107",NaN,NaN,NaN,11.0,40.861358,-73.897731,"(40.86135789900004, -73.89773096099998)",PATROL BORO BRONX,F

In [ ]:
#Loading Turnstile
bigfile = pd.read_csv(main_path + '/RiderData/TurnstileSmaller.csv', low_memory=False)

In [58]:
#Creating Ident
bigfile['Ident']= bigfile['Station2'].astype(str) + bigfile['year'].astype(str) + bigfile['week'].astype(str) + bigfile['wkdaynbr'].astype(str) + bigfile['timegroup'].astype(str)
bigfile[bigfile['Ident']==bigfile['Ident'].iloc[0]]

,Id,c.a,unit,station,linename,division,date,time,entries,exits,datetime,datetime_clean,year,weekday,group,group_id,station_line,entries_diff,exit_diff,week,hour,timegroup,timegroupstr,wkdaynbr,Station2,Longitude2,Latitude2,Ident
0,4373656,R236,R045,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,5442355,775583,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R236R04500-06-00GRD CNTRL-42 ST4567SIRT2019,0,GRD CNTRL-42 ST4567S,0,0,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121
154300,4373670,R240,R047,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,4981770,11531059,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R240R04700-00-00GRD CNTRL-42 ST4567SIRT2019,131,GRD CNTRL-42 ST4567S,57,359,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121
649200,4373676,R241A,R048,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,13150810,6487359,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R241AR04800-00-02GRD CNTRL-42 ST4567SIRT2019,527,GRD CNTRL-42 ST4567S,48,4,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121
2111023,4373665,R240,R047,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,37722626,8226483,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R240R04700-03-08GRD CNTRL-42 ST4567SIRT2019,1769,GRD CNTRL-42 ST4567S,101,20,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121
2130363,4373680,R241A,R048,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,627264,59652,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R241AR04800-00-03GRD CNTRL-42 ST4567SIRT2019,1788,GRD CNTRL-42 ST4567S,15,2,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121
3160257,4373649,R236,R045,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,2308424,986086,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R236R04500-03-01GRD CNTRL-42 ST4567SIRT2019,2642,GRD CNTRL-42 ST4567S,0,1,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121
3533291,4373671,R240,R047,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,5583529,21949551,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R240R04700-03-03GRD CNTRL-42 ST4567SIRT2019,2979,GRD CNTRL-42 ST4567S,9,43,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121
4921012,4373662,R237B,R047,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,582923,169949,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R237BR04701-00-02GRD CNTRL-42 ST4567SIRT2019,4144,GRD CNTRL-42 ST4567S,93,22,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121
5028200,4373666,R240,R047,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,304870,362499,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R240R04700-00-01GRD CNTRL-42 ST4567SIRT2019,4228,GRD CNTRL-42 ST4567S,91,237,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121
5544173,4373658,R236,R045,GRD CNTRL-42 ST,4567S,IRT,1/1/2019,4:00:00,1763207,4877774,01/01/2019 04:00:00,1/1/2019 4:00:00,2019,3,R236R04500-00-00GRD CNTRL-42 ST4567SIRT2019,4636,GRD CNTRL-42 ST4567S,7,22,1,4,1,12am - 4am,2,GRD CNTRL-42 ST_4567S,-73.99817432,40.604677,GRD CNTRL-42 ST_4567S2019121


In [48]:
#Creating a dataframe grouped by Ident
ident_sums=bigfile.groupby('Ident')['entries_diff','exit_diff'].sum().reset_index()
everything_else = bigfile.groupby('Ident').first()
bigfile2 = pd.merge(ident_sums, everything_else, on=['Ident'], how='left')

In [30]:
ident_sums.shape

(2310745, 3)

In [37]:
bigfile2.shape

(2310745, 32)

In [51]:
#Merging the Turnstile and Police Data
bigfile3 = pd.merge(bigfile2, dfcomb3, on=['Ident'], how='left')

In [68]:
bigfile3.sort_values(by=['Complaint_x'], ascending=False)

,Ident,entries_diff_x,exit_diff_x,Id,c.a,unit,station,linename,division,date,time,entries,exits,datetime_x,datetime_clean_x,year_x,weekday_x,group,group_id,station_line,entries_diff_y,exit_diff_y,week_x,hour_x,timegroup_x,timegroupstr_x,wkdaynbr_x,Station2,Longitude2,Latitude2,Complaint_x,OFNS_DESC_x,LAW_CAT_CD_x,Unnamed: 0,Unnamed: 0.1,CMPLNT_NUM,CMPLNT_FR_DT,DayofWeek,Month,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC_y,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD_y,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,ORIGINALROW,OBSOLETE_ID,LONG/LAT,Station,Distance,STATION_NAME.1,datetime_y,datetime_clean_y,year_y,weekday_y,week_y,hour_y,timegroup_y,timegroupstr_y,wkdaynbr_y,Complaint_y
1582789,HOYT-SCHER_ACG20182255,4088,2437,10258945,N108,R217,HOYT-SCHER,ACG,IND,6/1/2018,20:00:00,9452751,10909776,06/01/2018 20:00:00,6/1/2018 20:00:00,2018,6,N108R21700-00-04HOYT-SCHERACGIND2018,420,HOYT-SCHERACG,898,793,22,20,5,4pm - 8pm,5,HOYT-SCHER_ACG,-73.978189,40.592465,5.0,"FELONY ASSAULT, ASSAULT 3 & RELATED OFFENSES, ...","FELONY, MISDEMEANOR, MISDEMEANOR, MISDEMEANOR,...",11895.0,9977.0,704809348.0,6/1/2018,Fri,June,17:40:00,6/1/2018,17:50:00,84.0,6/1/2018,106.0,FELONY ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",COMPLETED,FELONY,BROOKLYN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1.0,NaN,NaN,NaN,"988,089","190,305",25-44,WHITE HISPANIC,M,30.0,40.689027,-73.986159,"(40.689027417000034, -73.98615855199995)",PATROL BORO BKLYN NORTH,HOYT-SCHERMERHORN,<18,BLACK,M,6.975694444,680775.0,"(-73.98615855, 40.68902742)",HOYT-SCHER_ACG,0.073,HOYT-SCHERMERHORN,6/1/2018 17:40:00,2018-06-01 17:40:00,2018.0,Friday,22.0,17.0,5.0,4pm - 8pm,5.0,1.0
676300,72 ST_1232018641,503,413,13231524,R161A,R452,72 ST,123,IRT,2/8/2018,4:00:00,1227804285,1923950951,02/08/2018 04:00:00,2/8/2018 4:00:00,2018,5,R161AR45201-06-0272 ST123IRT2018,3241,72 ST123,79,68,6,4,1,12am - 4am,4,72 ST_123,-73.93293256,40.66897831,4.0,"ASSAULT 3 & RELATED OFFENSES, ASSAULT 3 & RELA...","MISDEMEANOR, MISDEMEANOR, MISDEMEANOR, MISDEME...",6726.0,7759.0,992511572.0,2/8/2018,Thu,February,1:38:00,2/8/2018,2:18:00,20.0,2/8/2018,344.0,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,MANHATTAN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1.0,NaN,NaN,NaN,"989,219","223,014",25-44,BLACK,M,1.0,40.778805,-73.982060,"(40.778805016000035, -73.98205977899994)",PATROL BORO MAN NORTH,72 STREET,UNKNOWN,UNKNOWN,F,5.439583333,539601.0,"(-73.98205978, 40.77880502)",72 ST_123,0.025,72 STREET,2/8/2018 1:38:00,2018-02-08 01:38:00,2018.0,Thursday,6.0,1.0,1.0,12am - 4am,4.0,1.0
801479,96 ST_12320185116,2466,2941,5000792,R168A,R168,96 ST,123,IRT,12/17/2018,0:00:00,1203302,1331088,12/17/2018 00:00:00,12/17/2018 0:00:00,2018,2,R168AR16800-00-0096 ST123IRT2018,191,96 ST123,99,94,51,0,6,8pm - 12am,1,96 ST_123,-73.97209795,40.78134608,4.0,"OFFENSES AGAINST THE PERSON, FELONY ASSAULT, O...","MISDEMEANOR, FELONY, MISDEMEANOR, FELONY",7529.0,14166.0,565413652.0,12/17/2018,Mon,December,23:26:00,12/18/2018,0:23:00,24.0,12/18/2018,355.0,OFFENSES AGAINST THE PERSON,115.0,RECKLESS ENDANGERMENT 2,COMPLETED,MISDEMEANOR,MANHATTAN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1.0,NaN,NaN,NaN,"992,062","228,791",UNKNOWN,UNKNOWN,U,3.0,40.794659,-73.971788,"(40.79465928500008, -73.97178778199999)",PATROL BORO MAN NORTH,96 STREET,18-24,BLACK,M,1:14183,931935.0,"(-73.97178778, 40.79465929)",96 ST_123,0.058,96 STREET,12/17/2018 23:26:00,2018-12-17 23:26:00,2018.0,Monday,51.0,23.0,6.0,8pm - 12am,1.0,1.0
1418113,FOREST AVE_M20191122,52,112,2549661,K024,R403,FOREST AVE,M,BMT,3/12/2019,5:00:00,6269690,5780569,03/12/2019 05:00:00,3/12/2019 5:00:00,2019,3,K024R40300-00-01FOREST AVEMBMT2019,6753,FOREST AVEM,16,28,11,5,2,4am - 8am,2,FOREST AVE_M,-73.922156,40.689584,4.0,"CRIMINAL MISCHIEF & RELAT

In [53]:
#Saving Datasets
dfcomb3.to_csv(main_path + '/CrimesClean2.csv')
bigfile2.to_csv(main_path + '/ShrunkTurnstile.csv')
bigfile3.to_csv(main_path + '/Merged_Dataset2.csv')

In [56]:
bigfile3.sort_values(by=['Complaint_x_y'])

,Ident,entries_diff_x,exit_diff_x,Id,c.a,unit,station,linename,division,date,time,entries,exits,datetime_x,datetime_clean_x,year_x,weekday_x,group,group_id,station_line,entries_diff_y,exit_diff_y,week_x,hour_x,timegroup_x,timegroupstr_x,wkdaynbr_x,Station2,Longitude2,Latitude2,Complaint_x,OFNS_DESC_x,LAW_CAT_CD_x,Unnamed: 0,Unnamed: 0.1,CMPLNT_NUM,CMPLNT_FR_DT,DayofWeek,Month,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC_y,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD_y,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,ORIGINALROW,OBSOLETE_ID,LONG/LAT,Station,Distance,STATION_NAME.1,datetime_y,datetime_clean_y,year_y,weekday_y,week_y,hour_y,timegroup_y,timegroupstr_y,wkdaynbr_y,Complaint_y
175,1 AV_L20171432,1839,3664,15938398,H008,R248,1 AV,L,BMT,4/5/2017,8:00:00,6109038,646055,04/05/2017 08:00:00,4/5/2017 8:00:00,2017,4,H008R24801-00-041 AVLBMT2017,5655,1 AVL,253,181,14,8,2,4am - 8am,3,1 AV_L,-73.99107,40.730054,1.0,ASSAULT 3 & RELATED OFFENSES,MISDEMEANOR,12.0,1853.0,855410583.0,4/5/2017,Wed,April,8:04:00,4/5/2017,8:17:00,13.0,4/5/2017,344.0,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,MANHATTAN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1.0,NaN,NaN,NaN,"989,083","205,732",NaN,NaN,NaN,4.0,40.731370,-73.982563,"(40.73137039, -73.982563257)",PATROL BORO MAN SOUTH,1 AVENUE,25-44,BLACK,M,1.328472222,140285.0,"(-73.98256326, 40.73137039)",1 AV_L,0.054,1 AVENUE,4/5/2017 8:04:00,2017-04-05 08:04:00,2017.0,Wednesday,14.0,8.0,2.0,4am - 8am,3.0,1.0
1424280,FRANKLIN AV_2345S2017201,358,553,23147328,R622,R123,FRANKLIN AV,2345S,IRT,1/15/2017,3:00:00,303619268,101188718,01/15/2017 03:00:00,1/15/2017 3:00:00,2017,1,R622R12300-00-01FRANKLIN AV2345SIRT2017,3872,FRANKLIN AV2345S,55,40,2,3,1,12am - 4am,0,FRANKLIN AV_2345S,-73.91972,40.69866,1.0,GRAND LARCENY,FELONY,11117.0,243.0,508996870.0,1/15/2017,Sun,January,4:00:00,1/15/2017,4:30:00,77.0,1/16/2017,109.0,GRAND LARCENY,408.0,"LARCENY,GRAND FROM PERSON,LUSH WORKER(SLEEPING...",COMPLETED,FELONY,BROOKLYN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1.0,NaN,NaN,NaN,"995,908","183,618",NaN,NaN,NaN,32.0,40.670666,-73.957976,"(40.670666272, -73.957975908)",PATROL BORO BKLYN NORTH,FRANKLIN AVENUE,25-44,BLACK,M,0.213194444,44433.0,"(-73.95797591, 40.67066627)",FRANKLIN AV_2345S,0.009,FRANKLIN AVENUE,1/15/2017 4:00:00,2017-01-15 04:00:00,2017.0,Sunday,2.0,4.0,1.0,12am - 4am,0.0,1.0
1424326,FRANKLIN AV_2345S20172105,1540,1267,23146607,R622,R123,FRANKLIN AV,2345S,IRT,5/28/2017,20:00:00,303914055,101271464,05/28/2017 20:00:00,5/28/2017 20:00:00,2017,1,R622R12300-00-01FRANKLIN AV2345SIRT2017,3872,FRANKLIN AV2345S,282,99,21,20,5,4pm - 8pm,0,FRANKLIN AV_2345S,-73.91972,40.69866,3.0,"ASSAULT 3 & RELATED OFFENSES, ASSAULT 3 & RELA...","MISDEMEANOR, MISDEMEANOR, MISDEMEANOR",11133.0,2887.0,662618272.0,5/28/2017,Sun,May,19:55:00,5/28/2017,20:00:00,77.0,5/28/2017,344.0,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,COMPLETED,MISDEMEANOR,BROOKLYN,NaN,TRANSIT - NYC SUBWAY,N.Y. TRANSIT POLICE,1.0,NaN,NaN,NaN,"995,908","183,618",UNKNOWN,UNKNOWN,U,32.0,40.670666,-73.957976,"(40.670666272, -73.957975908)",PATROL BORO BKLYN NORTH,FRANKLIN AVENUE,25-44,WHITE HISPANIC,M,2.05,208269.0,"(-73.95797591, 40.67066627)",FRANKLIN AV_2345S,0.009,FRANKLIN AVENUE,5/28/2017 19:55:00,2017-05-28 19:55:00,2017.0,Sunday,21.0,19.0,5.0,4pm - 8pm,0.0,1.0
1424346,FRANKLIN AV_2345S20172134,2654,1387,23146632,R622,R123,FRANKLIN AV,2345S,IRT,5/24/2017,16:00:00,303906366,101269089,05/24/2017 16:00:00,5/24/2017 16:00:00,2017,4,R622R12300-00-01FRANKLIN AV2345SIRT2017,3872,FRANKLIN AV2345S,391,189,21,16,4,12pm - 4pm,3,FRANKLIN AV_2345S,-73.91972,40.69866,1.0,GRAND LARCENY,FELONY,11131.0,2808.0,571602095.0,5/24/2017,Wed,May,13:30:00,5/24/2017,13:45:00,77.0,5/24/2017,109.0,GRAND LARCENY,438.0,"LARCE

In [66]:
bigfile4 = bigfile3.groupby(['Ident'])['Complaint_y'].sum().reset_index()

In [70]:
#Adding Crime Descriptions in case of need for future use
bigfile3['LAW_CAT_CD_y']=bigfile3['LAW_CAT_CD_y'].astype(str)
bigfile3['OFNS_DESC']=bigfile3['OFNS_DESC'].astype(str)
bigfile3['PD_DESC']=bigfile3['PD_DESC'].astype(str)

In [73]:
PDDesc = bigfile3.groupby(['Ident'])['LAW_CAT_CD_y'].apply(', '.join).reset_index()

In [76]:
OffDesc = bigfile3.groupby(['Ident'])['OFNS_DESC'].apply(', '.join).reset_index()

In [77]:
LawCat = bigfile3.groupby(['Ident'])['LAW_CAT_CD_y'].apply(', '.join).reset_index()

In [79]:
bigfile5 = pd.merge(bigfile4, PDDesc, on=['Ident'], how='left')

In [80]:
bigfile5 = pd.merge(bigfile5, OffDesc, on=['Ident'], how='left')

In [81]:
bigfile5 = pd.merge(bigfile5, LawCat, on=['Ident'], how='left')

In [82]:
bigfile6 = pd.merge(bigfile5, bigfile[['Ident', 'Complaint3', 'LAW_CAT_CD']], left_on=['Ident'], right_on=['Ident'], how='left')

,Ident,Complaint_y,LAW_CAT_CD_y_x,OFNS_DESC,LAW_CAT_CD_y_y
0,1 AV_L20165262,0.0,nan,nan,nan
1,1 AV_L20165263,0.0,nan,nan,nan
2,1 AV_L20165264,0.0,nan,nan,nan
3,1 AV_L20165265,0.0,nan,nan,nan
4,1 AV_L20165266,0.0,nan,nan,nan
5,1 AV_L20171001,0.0,nan,nan,nan
6,1 AV_L20171002,0.0,nan,nan,nan
7,1 AV_L20171003,0.0,nan,nan,nan
8,1 AV_L20171004,0.0,nan,nan,nan
9,1 AV_L20171005,0.0,nan,nan,nan


In [84]:
everything_else2 = bigfile3.groupby('Ident').first()

In [86]:
bigfile5.shape

(2310745, 5)

In [87]:
everything_else2.shape

(2310745, 86)

In [88]:
bigfile6 = pd.merge(bigfile5, everything_else2, left_on=['Ident'], right_on=['Ident'], how='left')

In [89]:
bigfile6.head()

,Ident,Complaint_y_x,LAW_CAT_CD_y_x,OFNS_DESC_x,LAW_CAT_CD_y_y,entries_diff_x,exit_diff_x,Id,c.a,unit,station,linename,division,date,time,entries,exits,datetime_x,datetime_clean_x,year_x,weekday_x,group,group_id,station_line,entries_diff_y,exit_diff_y,week_x,hour_x,timegroup_x,timegroupstr_x,wkdaynbr_x,Station2,Longitude2,Latitude2,Complaint_x,LAW_CAT_CD_x,Unnamed: 0,Unnamed: 0.1,CMPLNT_NUM,CMPLNT_FR_DT,DayofWeek,Month,CMPLNT_FR_TM,...,OFNS_DESC_y,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD_y,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,ORIGINALROW,OBSOLETE_ID,LONG/LAT,Station,Distance,STATION_NAME.1,datetime_y,datetime_clean_y,year_y,weekday_y,week_y,hour_y,timegroup_y,timegroupstr_y,wkdaynbr_y,Complaint_y_y
0,1 AV_L20165262,0.0,nan,nan,nan,392,566,15921754,H007,R248,1 AV,L,BMT,12/31/2016,7:00:00,11892926,13050308,12/31/2016 07:00:00,12/31/2016 7:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,60,164,52,7,2,4am - 8am,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 AV_L20165263,0.0,nan,nan,nan,1499,1880,15921753,H007,R248,1 AV,L,BMT,12/31/2016,11:00:00,11893253,13050807,12/31/2016 11:00:00,12/31/2016 11:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,327,499,52,11,3,8am - 12pm,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 AV_L20165264,0.0,nan,nan,nan,3177,3329,15921752,H007,R248,1 AV,L,BMT,12/31/2016,15:00:00,11893870,13051562,12/31/2016 15:00:00,12/31/2016 15:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,617,755,52,15,4,12pm - 4pm,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1 AV_L20165265,0.0,nan,nan,nan,3915,5305,15921751,H007,R248,1 AV,L,BMT,12/31/2016,19:00:00,11894473,13052475,12/31/2016 19:00:00,12/31/2016 19:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,603,913,52,19,5,4pm - 8pm,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1 AV_L20165266,0.0,nan,nan,nan,4762,5172,15921750,H007,R248,1 AV,L,BMT,12/31/2016,23:00:00,11895162,13053430,12/31/2016 23:00:00,12/31/2016 23:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,689,955,52,23,6,8pm - 12am,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
#Combining Entries and Exits
bigfile6["EntExitCom"] = (bigfile6["entries_diff_x"] + bigfile6["exit_diff_x"])

In [92]:
#Calculating Crime Rate
bigfile6["CrimeRate"] = bigfile6["Complaint_y_x"]/bigfile6["EntExitCom"]

In [98]:
#Removing impossible values
bigfile6.replace([np.inf, -np.inf], np.nan)

,Ident,Complaint_y_x,LAW_CAT_CD_y_x,OFNS_DESC_x,LAW_CAT_CD_y_y,entries_diff_x,exit_diff_x,Id,c.a,unit,station,linename,division,date,time,entries,exits,datetime_x,datetime_clean_x,year_x,weekday_x,group,group_id,station_line,entries_diff_y,exit_diff_y,week_x,hour_x,timegroup_x,timegroupstr_x,wkdaynbr_x,Station2,Longitude2,Latitude2,Complaint_x,LAW_CAT_CD_x,Unnamed: 0,Unnamed: 0.1,CMPLNT_NUM,CMPLNT_FR_DT,DayofWeek,Month,CMPLNT_FR_TM,...,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD_y,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,ORIGINALROW,OBSOLETE_ID,LONG/LAT,Station,Distance,STATION_NAME.1,datetime_y,datetime_clean_y,year_y,weekday_y,week_y,hour_y,timegroup_y,timegroupstr_y,wkdaynbr_y,Complaint_y_y,EntExitCom,CrimeRate
0,1 AV_L20165262,0.0,nan,nan,nan,392,566,15921754,H007,R248,1 AV,L,BMT,12/31/2016,7:00:00,11892926,13050308,12/31/2016 07:00:00,12/31/2016 7:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,60,164,52,7,2,4am - 8am,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,958,0.0
1,1 AV_L20165263,0.0,nan,nan,nan,1499,1880,15921753,H007,R248,1 AV,L,BMT,12/31/2016,11:00:00,11893253,13050807,12/31/2016 11:00:00,12/31/2016 11:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,327,499,52,11,3,8am - 12pm,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3379,0.0
2,1 AV_L20165264,0.0,nan,nan,nan,3177,3329,15921752,H007,R248,1 AV,L,BMT,12/31/2016,15:00:00,11893870,13051562,12/31/2016 15:00:00,12/31/2016 15:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,617,755,52,15,4,12pm - 4pm,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6506,0.0
3,1 AV_L20165265,0.0,nan,nan,nan,3915,5305,15921751,H007,R248,1 AV,L,BMT,12/31/2016,19:00:00,11894473,13052475,12/31/2016 19:00:00,12/31/2016 19:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,603,913,52,19,5,4pm - 8pm,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9220,0.0
4,1 AV_L20165266,0.0,nan,nan,nan,4762,5172,15921750,H007,R248,1 AV,L,BMT,12/31/2016,23:00:00,11895162,13053430,12/31/2016 23:00:00,12/31/2016 23:00:00,2016,7,H007R24800-00-001 AVLBMT2016,1217,1 AVL,689,955,52,23,6,8pm - 12am,6,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9934,0.0
5,1 AV_L20171001,0.0,nan,nan,nan,2081,1676,15938501,H008,R248,1 AV,L,BMT,3/12/2017,4:00:00,6019948,636020,03/12/2017 04:00:00,3/12/2017 4:00:00,2017,1,H008R24801-00-041 AVLBMT2017,5655,1 AVL,578,29,10,4,1,12am - 4am,0,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,nan,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3757,0.0
6,1 AV_L20171002,0.0,nan,nan,nan,484,676,15938500,H008,R248,1 AV,L,BMT,3/12/2017,8:00:00,6020060,636038,03/12/2017 08:00:00,3/12/2017 8:00:00,2017,1,H008R24801-00-041 AVLBMT2017,5655,1 AVL,112,18,10,8,2,4am - 8am,0,1 AV_L,-73.99107,40.730054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,na

In [100]:
bigfile6.loc[~np.isfinite(bigfile6['CrimeRate']), 'CrimeRate'] = np.nan

In [101]:
bigfile6["CrimeRate"].mean()

9.77399423728436e-06

In [103]:
#Creating a Just Entry Crime Rate
bigfile6["CrimeRateJustEnt"] = bigfile6["Complaint_y_x"]/bigfile6["entries_diff_x"]

In [104]:
bigfile6.loc[~np.isfinite(bigfile6['CrimeRateJustEnt']), 'CrimeRateJustEnt'] = np.nan

In [105]:
bigfile6.to_csv(main_path + '/MergedDataUse.csv')

In [6]:
bigfile6.head()

,Unnamed: 0,Ident,Complaint_y_x,LAW_CAT_CD_y_x,OFNS_DESC_x,LAW_CAT_CD_y_y,entries_diff_x,exit_diff_x,Id,c.a,...,weekday_y,week_y,hour_y,timegroup_y,timegroupstr_y,wkdaynbr_y,Complaint_y_y,EntExitCom,CrimeRate,CrimeRateJustEnt
0,0,1 AV_L20165262,0.0,NaN,NaN,NaN,392,566,15921754,H007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,958,0.0,0.0
1,1,1 AV_L20165263,0.0,NaN,NaN,NaN,1499,1880,15921753,H007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3379,0.0,0.0
2,2,1 AV_L20165264,0.0,NaN,NaN,NaN,3177,3329,15921752,H007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6506,0.0,0.0
3,3,1 AV_L20165265,0.0,NaN,NaN,NaN,3915,5305,15921751,H007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9220,0.0,0.0
4,4,1 AV_L20165266,0.0,NaN,NaN,NaN,4762,5172,15921750,H007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9934,0.0,0.0
